### Hito3 para Determinantes del ingreso
*"Se generará un análisis descriptivo considerando el tipo de variables (desde el punto de vista estadístico así como computacional). Esta sección debe considerar medidas univariadas/ frecuencias, datos perdidos y gráficos distributivos sobre las variables a analizar. A partir de ésta se debe clarificar la estrategia de preprocesamiento (datos perdidos, recodificaciones)."*

###  Modelación descriptiva

Transformacion de variables categoricas con Binary Encoding.

In [1]:
%matplotlib inline

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as stats

import statsmodels.formula as sm
import statsmodels.formula.api as smf

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_curve, classification_report 

from graficos import *
import funciones as fun

import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('income_ready.csv')
print(df.shape)
df.head()

(48842, 15)


,age,workclass_recod,fnlwgt,educ_recod,educational-num,civstatus,collars,relationship,race,gender,capital-gain,capital-loss,hours-per-week,region,income
0,25,private,226802,high-school,7,never-married,blue-collar,Own-child,Black,Male,0,0,40,America,0
1,38,private,89814,high-school,9,married,blue-collar,Husband,White,Male,0,0,50,America,0
2,28,state-level-gov,336951,college,12,married,blue-collar,Husband,White,Male,0,0,40,America,1
3,44,private,160323,college,10,married,blue-collar,Husband,Black,Male,7688,0,40,America,1
4,18,NaN,103497,college,10,never-married,NaN,Own-child,White,Female,0,0,30,America,0


###  Modelación descriptiva

- Transformacion de variables categoricas con Binary Encoding.
- Se resetea el indice del dataframe


In [3]:
df_dummies = pd.get_dummies(df.dropna(), drop_first=True).reset_index(drop=True)
df_dummies.columns = [col.replace('-', '_') for col in df_dummies.columns]
df_dummies.head()


,age,fnlwgt,educational_num,capital_gain,capital_loss,hours_per_week,income,workclass_recod_private,workclass_recod_self_employed,workclass_recod_state_level_gov,...,relationship_Unmarried,relationship_Wife,race_Asian_Pac_Islander,race_Black,race_Other,race_White,gender_Male,region_Asia,region_Europe,region_Unknown
0,25,226802,7,0,0,40,0,1,0,0,...,0,0,0,1,0,0,1,0,0,0
1,38,89814,9,0,0,50,0,1,0,0,...,0,0,0,0,0,1,1,0,0,0
2,28,336951,12,0,0,40,1,0,0,1,...,0,0,0,0,0,1,1,0,0,0
3,44,160323,10,7688,0,40,1,1,0,0,...,0,0,0,1,0,0,1,0,0,0
4,34,198693,6,0,0,30,0,1,0,0,...,0,0,0,0,0,1,1,0,0,0


Se estima el modelo econométricos no lineal binario con Logit, usando df_dummies:

In [5]:
base = fun.mf(df_dummies, var_obj='income')             # Uso de la funcion para construir la base del modelo
model_logit = smf.logit(base, data=df_dummies).fit()    # Ajuste del modelo
model_logit.summary()

Optimization terminated successfully.
         Current function value: 0.332089
         Iterations 11


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                 income   No. Observations:                46033
Model:                          Logit   Df Residuals:                    45999
Method:                           MLE   Df Model:                           33
Date:                Fri, 08 Jul 2022   Pseudo R-squ.:                  0.4073
Time:                        21:21:41   Log-Likelihood:                -15287.
converged:                       True   LL-Null:                       -25791.
Covariance Type:            nonrobust   LLR p-value:                     0.000
===================================================================================================
                                      coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------------------
Intercept                          -7.6740      0.309    -24.805      0.000      -8.280      -7.068
age                                 0.0254      0.001     18.931      0.000       0.023       0.028
fnlwgt                            7.33e-07   1.39e-07      5.280      0.000    4.61e-07    1.01e-06
educational_num                     0.2715      0.016     17.171      0.000       0.241       0.302
capital_gain                        0.0003   8.56e-06     37.179      0.000       0.000       0.000
capital_loss                        0.0007   3.05e-05     21.720      0.000       0.001       0.001
hours_per_week                      0.0296      0.001     22.654      0.000       0.027       0.032
workclass_recod_private            -0.4532      0.074     -6.102      0.000      -0.599      -0.308
workclass_recod_self_employed      -0.7927      0.081     -9.753      0.000      -0.952      -0.633
workclass_recod_state_level_gov    -0.5268      0.083     -6.349      0.000      -0.689      -0.364
workclass_recod_unemployed         -1.5592      0.803     -1.942      0.052      -3.133       0.014
educ_recod_elementary_school        0.1688      0.229      0.738      0.461      -0.280       0.617
educ_recod_high_school             -0.0776      0.046     -1.674      0.094      -0.168       0.013
educ_recod_preschool               -4.0234      4.039     -0.996      0.319     -11.939       3.892
educ_recod_university               0.0907      0.060      1.509      0.131      -0.027       0.209
civstatus_married                   0.7768      0.136      5.715      0.000       0.510       1.043
civstatus_never_married            -0.4767      0.071     -6.669      0.000      -0.617      -0.337
civstatus_separated                -0.0533      0.133     -0.401      0.688      -0.313       0.207
civstatus_widowed                   0.0791      0.127      0.621      0.535      -0.170       0.329
collars_others                     -0.7209      0.085     -8.452      0.000      -0.888      -0.554
collars_white_collar                0.6021      0.035     17.176      0.000       0.533       0.671
relationship_Not_in_family         -0.9811      0.131     -7.477      0.000      -1.238      -0.724
relationship_Other_relative        -1.4663      0.184     -7.957      0.000      -1.828      -1.105
relationship_Own_child             -1.9446      0.163    -11.900      0.000      -2.265      -1.624
relationship_Unmarried             -1.2129      0.148     -8.196      0.000      -1.503      -0.923
relationship_Wife                   1.0555      0.083     12.679      0.000       0.892       1.219
race_Asian_Pac_Islander             0.6153      0.214      2.872      0.004       0.195       1.035
race_Black                          0.2602      0.189      1.379      0.168      -0.110       0.630
race_Other                          0.1926      0.268      0.719      0.472      -0.333       0.718
race_White                          0.513